In [1]:
import nltk
import json


In [2]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
nltk.download('punkt')  # Download the necessary tokenizer
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
with open(r"/content/BOOK2.txt", "r") as file:
    text = file.read()

In [5]:
# Split the text into sentences
sentences = sent_tokenize(text)

In [6]:
chunk_size = 500
chunks = [sentences[i:i + chunk_size] for i in range(0, len(sentences), chunk_size)]


In [7]:
chunks = [' '.join(chunk) for chunk in chunks]

In [8]:
from transformers import T5ForConditionalGeneration , T5Tokenizer
model_name = "t5-base"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [9]:
def generate_question(text):
    # Prepare the text for question generation
    input_text = f"generate question: {text}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate the question
    outputs = model.generate(inputs['input_ids'], max_length=64, num_beams=5, early_stopping=True)

    # Decode and return the question
    question = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return question

# Generate questions for each chunk
questions = [generate_question(chunk) for chunk in chunks]

In [10]:
from transformers import pipeline

In [11]:
#load question answere pipeline
qa_pipeline = pipeline("question-answering")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu


In [12]:
def get_answer(question, context):
    # Use the QA model to extract the answer from the context (chunk of text)
    result = qa_pipeline(question=question, context=context)
    return result['answer']

# Generate answers for each question
qa_pairs = []
for question, context in zip(questions, chunks):
    answer = get_answer(question, context)
    qa_pairs.append({
        "question": question,
        "answer": answer,
        "context": context
    })

In [13]:
import json

# Save the generated QA pairs in SQuAD-like format
qa_data = {
    "data": [
        {
            "title": "Man Thinkter",
            "paragraphs": [
                {
                    "context": chunk,
                    "qas": [
                        {
                            "question": qa["question"],
                            "id": f"qa-{i}",
                            "answers": [
                                {
                                    "text": qa["answer"],
                                    "answer_start": chunk.find(qa["answer"])
                                }
                            ]
                        }
                    ]
                }
                for i, (qa, chunk) in enumerate(zip(qa_pairs, chunks))
            ]
        }
    ]
}

# Write to a JSON file
with open("book_qa_SLM.json", "w") as json_file:
    json.dump(qa_data, json_file, indent=2)


In [14]:
for qa_pair in qa_pairs[:9]:  # Print the first 5 QA pairs

    print(f"Question: {qa_pair['question']}")
    print(f"Answer: {qa_pair['answer']}")
    print()


Question: True
Answer: Project Gutenberg™ eBooks

